In [1]:
from Load_Data_for_Modelling_Function import Data_for_Model
from Load_Data_for_Modelling_Interpolation import Interpolation

data = Data_for_Model(0,1800)

Anzahl der Zeilen im finalen DataFrame: 255600
None


In [2]:
from Fensterung_Scaling_CNN_ValData_Interpolation_Test import Fensterung_Scale
X_train_scaled, X_val_scaled, X_test_scaled, Y_train_scaled, Y_val_scaled, Y_test_scaled, Y_train, Y_val, Y_test, scalers_features, scaler_labels, Angepasste_Blechnummern_test = Fensterung_Scale(data, Validation_data=1, random=42, Train_Test_Split=2, window_size=10)

# Falls Validations Daten benötigt werden
#X_train_scaled, X_val_scaled, X_test_scaled, Y_train_scaled, Y_val_scaled, Y_test_scaled, Y_train, Y_val, Y_test, scalers_features, scaler_labels, Blechnummern, Test_Blechnummern = Fensterung_Scale(data, Validation_data=1, random=42, Train_Test_Split=2, window_size=10)
# Falls keine Validationsdaten benötigt werden 
#X_train_scaled, X_test_scaled, Y_train_scaled, Y_test_scaled, Y_train, Y_test, scalers_features, scaler_labels = Fensterung_Scale(Validation_data=0, random=42)

142
254180
142.0
Richtige Blechnummern umgerechnet der Validierungsdaten [42, 20, 84, 76, 71, 49, 40, 36, 135, 22, 21, 37, 69, 73]
Richtige Blechnummern umgerechnet der Testdaten [78, 103, 17, 85, 39, 117, 109, 115, 83, 67, 169, 165, 101, 164]
Blechnummern für Validierungsdaten, abgezählt vom Array nicht die Originaldaten [29, 7, 71, 63, 58, 36, 27, 23, 109, 9, 8, 24, 56, 60]
Blechnummern für Testdaten, abgezählt vom Array nicht die Originaldaten[65, 78, 4, 72, 26, 92, 84, 90, 70, 54, 141, 137, 76, 136]
25060
Shape nach dem Random Sampling des Arrays von X_val: (25060, 10, 11)
Shape nach dem Random Sampling des Arrays von X_train: (204060, 10, 11)
Shape nach dem Random Sampling des Arrays von Y_train: (204060, 1, 3)
(25060, 1, 3)
Shape für die Features der gesamten Trainingsdaten, also im Falle einer Interpolation mit allen Daten (204060, 10, 11)
Shape für die Features der gesamten Testdaten, also im Falle einer Interpolation mit allen Daten (25060, 10, 11) 
Shape für die Labels der ge

In [3]:
import numpy as np
Y_train = np.squeeze(Y_train)
Y_test = np.squeeze(Y_test)
Y_val =np.squeeze(Y_val)
Y_train_scaled = np.squeeze(Y_train_scaled)
Y_val_scaled = np.squeeze(Y_val_scaled)

In [ ]:
print(X_train_scaled)
print(Y_train)
print(Y_val)
print(X_val_scaled)

In [4]:
# Modellierung LSTM 

# Aktuell bestes Modell aufbauen 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from keras import layers
import matplotlib.pyplot as plt
import kerastuner as kt
import shutil
import os
import json
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from tensorflow.keras.layers import LSTM, Dense

# Remove the previous tuner directory if it exists
tuner_directory = 'my_dir'
if os.path.exists(tuner_directory):
    shutil.rmtree(tuner_directory)

def build_model(hp):
    # CNN Modell definieren
    input_layer = layers.Input(shape=(10,11))

     #Aktivierungsfunktionen
    # hp_activation_lstm_0 = hp.Choice('activation_11', values =['relu','tanh'])
    #hp_recurrent_activation_lstm_0 = hp.Choice('activation_recurrent_11', values =['relu','tanh','sigmoid'])
    
    # Hyperparameter for the number of units in the hidden layer
    # lstm_filter = hp.Int('layer1', min_value=32, max_value=512, step=32)
    
    
    num_layers_lstm = hp.Int('num_layers_lstm', 0, 6)
    print(f'Anzahl an LSTM Layers: {num_layers_lstm}')
    # Hyperparameter for the learning rate
    learning_rate = hp.Float('learning_rate', min_value=1e-5, max_value=1e-2, sampling='LOG')
    
    # Hyperparameter für Regularisierungen
    #l1_reg = hp.Float('l1', min_value=0.0, max_value=0.01, step=0.001)
    # l2_reg = hp.Float('l2', min_value=0.0, max_value=0.01, step=0.001)

    #kernel_regularizer=keras.regularizers.l1_l2(l1=l1_reg, l2=l2_reg)
    #, kernel_regularizer=keras.regularizers.l2(l2_reg)
    
    x = input_layer

    
    for i in range(num_layers_lstm):
        units_lstm = hp.Int(f'units_lstm{i}', min_value=32, max_value=512, step=32)
        hp_activation_lstm = hp.Choice(f'activation_lstm{i}', values =['relu','tanh','sigmoid'])
        #hp_recurrent_activation_lstm = hp.Choice(f'activation_recurrent_lstm{i}', values =['relu','tanh','sigmoid'])
        #l1_regulizer= hp.Float(f'l2{i}', min_value=0.0, max_value=0.01, step=0.001)
        l2_regulizer =hp.Float(f'l2_conv{i}', min_value=0.0, max_value=0.01, step=0.001)
    
        #, kernel_regularizer=keras.regularizers.l2(l2_regulizer)
        if i == num_layers_lstm - 1:
            x = layers.SimpleRNN(units_lstm, activation=hp_activation_lstm, return_sequences=False, kernel_regularizer=keras.regularizers.l2(l2_regulizer))(x)
        else:
            x = layers.SimpleRNN(units_lstm, activation=hp_activation_lstm, return_sequences=True,  kernel_regularizer=keras.regularizers.l2(l2_regulizer))(x)
    

    num_layers_fully = hp.Int('num_layers_fully', 1, 6)
    hp_layer_1 = hp.Int('layer_dense', min_value=32, max_value=512, step=32)
    hp_activation_fully1 = hp.Choice('activation_fully', values =['relu','tanh','sigmoid'])
    print(f'Anzahl an Fully Connected Layers: {num_layers_fully}')
    #kernel_regularizer=keras.regularizers.l2(l2_dense)
    l2_dense = hp.Float('l2_dense', min_value=0.0, max_value=0.01, step=0.001)
    # Fully Connected Layers
    dense_layer = layers.Dense(hp_layer_1, activation=hp_activation_fully1, kernel_regularizer=keras.regularizers.l2(l2_dense))(x)
    
    for i in range(num_layers_fully-1):
        units_dense = hp.Int(f'units_dense{i}', min_value=32, max_value=512, step=32)
        activation_layer_dense = hp.Choice(f'activation_dense{i}', values =['relu','tanh'])
        l2_dense_x = hp.Float(f'l2_dense{i}', min_value=0.0, max_value=0.01, step=0.001)
        #kernel_regularizer=keras.regularizers.l2(l2_dense_x) 
        dense_layer = layers.Dense(units_dense, activation=activation_layer_dense, kernel_regularizer=keras.regularizers.l2(l2_dense_x))(dense_layer)

    # Output Layers definieren
    X_output = layers.Dense(1, activation='linear', name='Verstellweg_X')(dense_layer)
    Y_output = layers.Dense(1, activation='linear', name='Verstellweg_Y')(dense_layer)
    Phi_output = layers.Dense(1, activation='linear', name='Verstellweg_Phi')(dense_layer)
    
    
    # Liste erstellen für alle Outputs
    outputs = [X_output, Y_output, Phi_output]

    # Modell definieren 
    model = keras.Model(inputs=input_layer, outputs=outputs)

    # Kompilieren des Modells
    model.compile(optimizer=keras.optimizers.Adam(learning_rate), 
                loss=['mean_absolute_error', 'mean_absolute_error', 'mean_absolute_error'], 
                metrics={'Verstellweg_X': 'mae', 'Verstellweg_Y': 'mae', 'Verstellweg_Phi': 'mae'})

    # Modell zusammenfassen
    model.summary()
    
    return model 

# Instantiate the Keras Tuner
tuner = kt.BayesianOptimization(build_model,
                     objective='val_loss',
                     max_trials=30,
                     executions_per_trial=1,
                     directory='my_dir',
                     project_name='RNN_Hyperparametertuning_002')

# Define a custom callback to stop training if validation loss is above a threshold after 2 epochs
class EarlyStopOnHighValLoss(tf.keras.callbacks.Callback):
    def __init__(self, threshold, patience=3):
        super(EarlyStopOnHighValLoss, self).__init__()
        self.threshold = threshold
        self.patience = patience
        self.wait = 0
        
    def on_epoch_end(self, epoch, logs=None):
        val_loss = logs.get('val_loss')
        if val_loss is not None and val_loss > self.threshold:
            self.wait += 1
            if self.wait >= self.patience:
                self.model.stop_training = True
        else:
            self.wait = 0


# Define early stopping to prevent overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

# Define the custom callback for early stopping on high validation loss
early_stop_on_high_val_loss = EarlyStopOnHighValLoss(threshold=2, patience=3)  

# Run the hyperparameter search
tuner.search(X_train_scaled, [Y_train[:, 0], Y_train[:, 1], Y_train[:, 2]],
             epochs=20,
             validation_data=(X_val_scaled, [Y_val[:, 0], Y_val[:, 1], Y_val[:, 2]]),
             callbacks=[early_stopping,early_stop_on_high_val_loss])


# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Speichere Hyperparameter in einer JSON Datei
best_hyperparameters = best_hps.values
Pfad = r'C:\Users\corvi\OneDrive - stud.tu-darmstadt.de\Desktop\Masterthesis\14_Modelle_Hyperparameter'
hyperparameters_pfad = os.path.join(Pfad, 'best_hyperparameters_RNN_Bayesian_30TrialsTest.json')

with open(hyperparameters_pfad, 'w') as json_file:
    json.dump(best_hyperparameters, json_file)

# Debugging: Print all hyperparameters
print("All available hyperparameters:")
print(best_hps.values)

print(f"Best activation LSTM1: {best_hps.get('activation_11')}")
#print(f"Best recurrent activation LSTM1: {best_hps.get('activation_recurrent_11')}")
print(f"Best layer1 units (filters): {best_hps.get('layer1')}")
print(f"Best number of LSTM layers: {best_hps.get('num_layers_lstm')}")
print(f"Best number of fully connected layers: {best_hps.get('num_layers_fully')}")
print(f"Best activation Fully1: {best_hps.get('activation_fully')}")
print(f"Best layer_dense units: {best_hps.get('layer_dense')}")
print(f"Best learning rate: {best_hps.get('learning_rate')}")

for i in range(best_hps.get('num_layers_lstm') - 1):
    print(f"Best units_lstm{i}: {best_hps.get(f'units_lstm{i}')}")
    print(f"Best activation_lstm{i}: {best_hps.get(f'activation_lstm{i}')}")
    #print(f"Best rekurrente activation_lstm{i}: {best_hps.get(f'activation_recurrent_lstm{i}')}")
    
for i in range(best_hps.get('num_layers_fully') - 1):
    print(f"Best units_dense{i}: {best_hps.get(f'units_dense{i}')}")
    print(f"Best activation_dense{i}: {best_hps.get(f'activation_dense{i}')}")

# Modell mit den besten Hyperparametern aufbauen trainieren und testen
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(X_train_scaled, [Y_train[:, 0], Y_train[:, 1], Y_train[:, 2]],
                         epochs=50,
                         validation_data=(X_test_scaled,[Y_test[:, 0], Y_test[:, 1], Y_test[:, 2]]),
                         callbacks=[early_stopping])

# Speichern des Modells
model_pfad = os.path.join(Pfad, 'best_model_RNN_30Trials.h5')
best_model.save(model_pfad)

# Plot validation loss for each output
plt.figure(figsize=(12, 8))
plt.plot(history.history['val_Verstellweg_X_mae'], label='Validation Loss - Verstellweg_X')
plt.plot(history.history['val_Verstellweg_Y_mae'], label='Validation Loss - Verstellweg_Y')
plt.plot(history.history['val_Verstellweg_Phi_mae'], label='Validation Loss - Verstellweg_Phi')
plt.title('Validation Loss for Each Output')
plt.xlabel('Epochs')
plt.ylabel('Validation Loss')

plt.legend()
plt.show()





Trial 30 Complete [00h 02m 09s]
val_loss: 3.1622440814971924

Best val_loss So Far: 1.5030701160430908
Total elapsed time: 03h 47m 27s
All available hyperparameters:
{'num_layers_lstm': 1, 'learning_rate': 0.0004426556642885296, 'units_lstm0': 64, 'activation_lstm0': 'sigmoid', 'l2_conv0': 0.001, 'num_layers_fully': 5, 'layer_dense': 224, 'activation_fully': 'relu', 'l2_dense': 0.009000000000000001, 'units_lstm1': 384, 'activation_lstm1': 'sigmoid', 'l2_conv1': 0.002, 'units_dense0': 224, 'activation_dense0': 'relu', 'l2_dense0': 0.0, 'units_dense1': 96, 'activation_dense1': 'tanh', 'l2_dense1': 0.007, 'units_lstm2': 96, 'activation_lstm2': 'tanh', 'l2_conv2': 0.004, 'units_lstm3': 288, 'activation_lstm3': 'sigmoid', 'l2_conv3': 0.003, 'units_lstm4': 160, 'activation_lstm4': 'relu', 'l2_conv4': 0.0, 'units_dense2': 384, 'activation_dense2': 'relu', 'l2_dense2': 0.005, 'units_lstm5': 128, 'activation_lstm5': 'relu', 'l2_conv5': 0.003, 'units_dense3': 32, 'activation_dense3': 'relu', 'l2

KeyError: 'activation_11 does not exist.'

In [5]:
from sklearn.metrics import mean_absolute_error
# Vorhersagen für den Testdatensatz erstellen
predictions = best_model.predict(X_test_scaled)
X_p, Y_p, Phi_p = predictions

 # MAE für den Testdatensatz berechnen
mae_X = mean_absolute_error(Y_test[:,0], X_p)
mae_Y = mean_absolute_error(Y_test[:,1], Y_p)
mae_Phi = mean_absolute_error(Y_test[:,2], Phi_p)

print(f"Mean Absolute Error for Verstellweg_X: {mae_X}")
print(f"Mean Absolute Error for Verstellweg_Y: {mae_Y}")
print(f"Mean Absolute Error for Verstellweg_Phi: {mae_Phi}")

784/784 ━━━━━━━━━━━━━━━━━━━━ 28s 34ms/step
Mean Absolute Error for Verstellweg_X: 0.013959367488342183
Mean Absolute Error for Verstellweg_Y: 0.60074614639255
Mean Absolute Error for Verstellweg_Phi: 0.5190988378086427


In [10]:
input_layer = layers.Input(shape=(10,11))

lstm_1 = layers.LSTM(32, activation='relu', return_sequences=True)(input_layer)
lstm_2 = layers.LSTM(64, activation='relu', return_sequences=True)(lstm_1)

dense_layer = layers.Dense(512, activation='relu')(lstm_2)
#dense_layer_1 = layers.Dense(32, activation='relu')(dense_layer)

# Output Layers definieren
X_output = layers.Dense(1, activation='linear', name='Verstellweg_X')(dense_layer)
Y_output = layers.Dense(1, activation='linear', name='Verstellweg_Y')(dense_layer)
Phi_output = layers.Dense(1, activation='linear', name='Verstellweg_Phi')(dense_layer)

# Liste erstellen für alle Outputs
outputs = [X_output, Y_output, Phi_output]

# Modell definieren 
model = keras.Model(inputs=input_layer, outputs=outputs)


# Modell zusammenfassen
model.summary()
# Kompilieren des Modells
model.compile(optimizer=keras.optimizers.Adam(0.001), 
            loss=['mean_absolute_error', 'mean_absolute_error', 'mean_absolute_error'], 
            metrics={'Verstellweg_X': 'mae', 'Verstellweg_Y': 'mae', 'Verstellweg_Phi': 'mae'})

# plot Model
model.summary()

LSTM_model = model.fit(X_train_scaled, [Y_train[:,0], Y_train[:,1], Y_train[:,2]],
                        epochs=15,
                        validation_data=(X_val_scaled, [Y_val[:, 0], Y_val[:, 1], Y_val[:, 2]]))


# Plot validation loss for each output
plt.figure(figsize=(12, 8))
plt.plot(LSTM_model.history['val_Verstellweg_X_mae'], label='Validation Loss - Verstellweg_X')
plt.plot(LSTM_model.history['val_Verstellweg_Y_mae'], label='Validation Loss - Verstellweg_Y')
plt.plot(LSTM_model.history['val_Verstellweg_Phi_mae'], label='Validation Loss - Verstellweg_Phi')
plt.title('Validation Loss for Each Output')
plt.xlabel('Epochs')
plt.ylabel('Validation Loss')

plt.legend()
plt.show()

# Vorhersagen für den Testdatensatz erstellen
predictions = model.predict(X_test_scaled)
X_p, Y_p, Phi_p = predictions

#MAE für den Testdatensatz berechnen
mae_X = mean_absolute_error(Y_test[:,0], X_p)
mae_Y = mean_absolute_error(Y_test[:,1], Y_p)
mae_Phi = mean_absolute_error(Y_test[:,2], Phi_p)
r2_X = r2_score(Y_test[:,0], X_p)
r2_Y =r2_score(Y_test[:,1], Y_p)
r2_phi =r2_score(Y_test[:,2], Phi_p)

print(f"Mean Absolute Error für Verstellweg_X: {mae_X}")
print(f"Mean Absolute Error für Verstellweg_Y: {mae_Y}")
print(f"Mean Absolute Error für Verstellweg_Phi: {mae_Phi}")

print(f"R2 für Verstellweg_X: {r2_X}")
print(f"R2 für Verstellweg_Y: {r2_Y}")
print(f"R2 für Verstellweg_Phi: {r2_phi}")

Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 10, 11)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_16 (LSTM)      │ (None, 10, 32)    │      5,632 │ input_layer_5[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_17 (LSTM)      │ (None, 10, 64)    │     24,832 │ lstm_16[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 10, 512)   │     33,280 │ lstm_17[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Verstellweg_X       │ (None, 10, 1)     │        513 │ dense_9[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Verstellweg_Y       │ (None, 10, 1)     │        513 │ dense_9[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Verstellweg_Phi     │ (None, 10, 1)     │        513 │ dense_9[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 65,283 (255.01 KB)

 Trainable params: 65,283 (255.01 KB)

 Non-trainable params: 0 (0.00 B)

...

 Total params: 65,283 (255.01 KB)

 Trainable params: 65,283 (255.01 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/15
6377/6377 ━━━━━━━━━━━━━━━━━━━━ 133s 19ms/step - Verstellweg_Phi_mae: 2.2899 - Verstellweg_X_mae: 0.2628 - Verstellweg_Y_mae: 2.4485 - loss: 5.0012 - val_Verstellweg_Phi_mae: 2.3751 - val_Verstellweg_X_mae: 0.1430 - val_Verstellweg_Y_mae: 3.6786 - val_loss: 6.1966
Epoch 2/15
6377/6377 ━━━━━━━━━━━━━━━━━━━━ 116s 18ms/step - Verstellweg_Phi_mae: 2.2880 - Verstellweg_X_mae: 0.2672 - Verstellweg_Y_mae: 2.4497 - loss: 5.0048 - val_Verstellweg_Phi_mae: 2.3750 - val_Verstellweg_X_mae: 0.1429 - val_Verstellweg_Y_mae: 3.6786 - val_loss: 6.1966
Epoch 3/15
6377/6377 ━━━━━━━━━━━━━━━━━━━━ 115s 18ms/step - Verstellweg_Phi_mae: 2.2877 - Verstellweg_X_mae: 0.2653 - Verstellweg_Y_mae: 2.4550 - loss: 5.0080 - val_Verstellweg_Phi_mae: 2.3750 - val_Verstellweg_X_mae: 0.1429 - val_Verstellweg_Y_mae: 3.6786 - val_loss: 6.1965
Epoch 4/15
6377/6377 ━━━━━━━━━━━━━━━━━━━━ 130s 20ms/step - Verstellweg_Phi_mae: 2.2829 - Verstellweg_X_mae: 0.2606 - Verstellweg_Y_mae: 2.4542 - loss: 4.9977 - val_Verstellweg

KeyboardInterrupt: 

In [1]:
# Aktuell bestes Modell aufbauen 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from keras import layers
import matplotlib.pyplot as plt
import kerastuner as kt
import shutil
import os
import json
import gc


def bestes_model(X_train_scaled, X_val_scaled, X_test_scaled, Y_train_scaled, Y_val_scaled, Y_test_scaled, Y_train, Y_val, Y_test, scalers_features, scaler_labels):
      
    Y_train = np.squeeze(Y_train)
    Y_test = np.squeeze(Y_test)
    Y_val =np.squeeze(Y_val)
    Y_train_scaled = np.squeeze(Y_train_scaled)
    Y_val_scaled = np.squeeze(Y_val_scaled)
    
    # TensorFlow-Sitzung zurücksetzen
    tf.keras.backend.clear_session()
    gc.collect()

# LSTM Modell definieren
    input_layer = layers.Input(shape=(10,11))

    # LSTM Layers
    # lstm_1 = layers.LSTM(32, activation='tanh', return_sequences=True, kernel_regularizer=keras.regularizers.l2(0.003))(input_layer)
    # lstm_2 = layers.LSTM(96, activation='relu', return_sequences=True, kernel_regularizer=keras.regularizers.l2(0.006))(lstm_1)
    # lstm_3 = layers.LSTM(224, activation='tanh', return_sequences=True, kernel_regularizer=keras.regularizers.l2(0.002))(lstm_2)
    # lstm_4 = layers.LSTM(224, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001))(lstm_3)
    
      # LSTM Layers Modell 2 30 Trials
    lstm_1 = layers.LSTM(128, activation='tanh', return_sequences=True, kernel_regularizer=keras.regularizers.l2(0.008))(input_layer)
    lstm_2 = layers.LSTM(416, activation='relu', return_sequences=True, kernel_regularizer=keras.regularizers.l2(0.001))(lstm_1)
    lstm_3 = layers.LSTM(416, activation='relu', return_sequences=True, kernel_regularizer=keras.regularizers.l2(0.003))(lstm_2)
    lstm_4 = layers.LSTM(224, activation='sigmoid', kernel_regularizer=keras.regularizers.l2(0.002))(lstm_3)

    # print(flatten)
    # print(type(flatten))
    # print(flatten.shape)

    # Fully Connected Layers
    dense_layer = layers.Dense(224, activation='relu', kernel_regularizer=keras.regularizers.l2(0))(lstm_4)
    dense_layer_1 = layers.Dense(160, activation='tanh', kernel_regularizer=keras.regularizers.l2(0.006))(dense_layer)
    dense_layer_2 = layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01))(dense_layer_1)
    dense_layer_3 = layers.Dense(288, activation='tanh', kernel_regularizer=keras.regularizers.l2(0.004))(dense_layer_2)
    
        # Fully Connected Layers Modell 2 30 Trials
    # dense_layer = layers.Dense(320, activation='tanh', kernel_regularizer=keras.regularizers.l2(0))(lstm_4)
        
    # Output Layers definieren
    X_output = layers.Dense(1, activation='linear', name='Verstellweg_X')(dense_layer_3)
    Y_output = layers.Dense(1, activation='linear', name='Verstellweg_Y')(dense_layer_3)
    Phi_output = layers.Dense(1, activation='linear', name='Verstellweg_Phi')(dense_layer_3)

    # Liste erstellen für alle Outputs
    outputs = [X_output, Y_output, Phi_output]

    # Modell definieren 
    model = keras.Model(inputs=input_layer, outputs=outputs)

    # Kompilieren des Modells1
    model.compile(optimizer=keras.optimizers.Adam(0.00014383567347209258), 
                loss=['mean_absolute_error', 'mean_absolute_error', 'mean_absolute_error'], 
                metrics={'Verstellweg_X': 'mae', 'Verstellweg_Y': 'mae', 'Verstellweg_Phi': 'mae'})
    
        # Kompilieren des Modells
    # model.compile(optimizer=keras.optimizers.Adam(0.0007952710448240707), 
    #             loss=['mean_absolute_error', 'mean_absolute_error', 'mean_absolute_error'], 
    #             metrics={'Verstellweg_X': 'mae', 'Verstellweg_Y': 'mae', 'Verstellweg_Phi': 'mae'})

    # Modell zusammenfassen
    #model.summary()

    # Define early stopping to prevent overfitting
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

    CNN = model.fit(X_train_scaled, [Y_train[:,0], Y_train[:,1], Y_train[:,2]],
                            epochs=20,
                            validation_data=(X_val_scaled, [Y_val[:, 0], Y_val[:, 1], Y_val[:, 2]]),
                            callbacks=[early_stopping])
    
    # Vorhersagen für den Testdatensatz erstellen
    predictions = model.predict(X_test_scaled)
    X_p, Y_p, Phi_p = predictions
    
    # print(X_p.shape)
    # print(X_p[:,0].shape)
    # print(Y_test[:,0].shape)
     # Error für jedes Label berechnen
    Fehler_X = Y_test[:,0]-X_p[:,0]
    Fehler_Y = Y_test[:,1]-Y_p[:,0]
    Fehler_Phi = Y_test[:,2]-Phi_p[:,0]
    
    # print(Fehler_X)
    # print(Fehler_X.shape)
    
    # Fehler in einen DataFrame konvertieren
    df_Fehler = pd.DataFrame({
        'Label_X': Y_test[:,0],
        'Label_Y': Y_test[:,1],
        'Label_Phi': Y_test[:,2],
        'Fehler_X': Fehler_X,
        'Fehler_Y': Fehler_Y,
        'Fehler_Phi': Fehler_Phi})
    
    mae_X = mean_absolute_error(Y_test[:, 0], X_p)
    mae_Y = mean_absolute_error(Y_test[:, 1], Y_p)
    mae_Phi = mean_absolute_error(Y_test[:, 2], Phi_p)

    # X und Y kombinieren da diese zusammen sakliert werden
    # XY_p = np.column_stack((X_p, Y_p))

    # Rückskalierung der Vorhersagen
    # XY_pred = scaler_Y_mm.inverse_transform(XY_p)
    # X_pred, Y_pred = XY_pred[:, 0], XY_pred[:, 1]
    # Phi_pred = scaler_Y_phi.inverse_transform(Phi_p.reshape(-1, 1)).flatten()
    
    # MAE für den Testdatensatz berechnen
    # mae_X = mean_absolute_error(Y_test[:,0], X_p)
    # mae_Y = mean_absolute_error(Y_test[:,1], Y_p)
    # mae_Phi = mean_absolute_error(Y_test[:,2], Phi_p)

    print(f"Mean Absolute Error for Verstellweg_X: {mae_X}")
    print(f"Mean Absolute Error for Verstellweg_Y: {mae_Y}")
    print(f"Mean Absolute Error for Verstellweg_Phi: {mae_Phi}")
    
    return mae_X, mae_Y, mae_Phi, df_Fehler

C:\Users\corvi\AppData\Local\Temp\ipykernel_9772\720250685.py:10: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [2]:
import random 
from Splitting_Scaling_Function_SkalierungY import Split_Scaling
from Load_Data_for_Modelling_Function import Data_for_Model
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from Fensterung_Scaling_CNN_ValData import Fensterung_Scale
from sklearn.metrics import mean_absolute_error
import pandas as pd


random.seed(2)
# Generieren einer Liste von 10 eindeutigen zufälligen Ganzzahlen zwischen 0 und 100
Random_numbers = random.sample(range(101), 10)
print(Random_numbers)

Random_numbers = Random_numbers[9:10]
print(Random_numbers)


MAE_StandardSplit_df = pd.DataFrame(columns=['CV', 'Datentyp','Error', 'X', 'y', 'phi'])
MAE_BlechSplit_df = pd.DataFrame(columns=['CV', 'Datentyp','Error', 'X', 'y', 'phi'])


data = Data_for_Model(0,1800)

Liste_Fehler_Blechsplit = []
Liste_Fehler_Standardsplit= []

Liste_MAEs_Blechsplit =[]
Liste_MAEs_Standardsplit = []

MAE_StandardSplit_leer = pd.DataFrame(columns=['CV', 'Datentyp','Error', 'X', 'y', 'phi'])
MAE_BlechSplit_leer = pd.DataFrame(columns=['CV', 'Datentyp','Error', 'X', 'y', 'phi'])

for n in Random_numbers:
    
    X_train, X_val, X_test, Y_train_scaled, Y_val_scaled, Y_test_scaled, Y_train, Y_val, Y_test, scalers_features, scaler_labels, Angepasste_Blechnummern_test = Fensterung_Scale(data, Validation_data=1, random=n, Train_Test_Split =1, size=0.2)
    mae_X, mae_Y, mae_phi, df_Fehler = bestes_model(X_train, X_val, X_test, Y_train_scaled, Y_val_scaled, Y_test_scaled, Y_train, Y_val, Y_test, scalers_features, scaler_labels)
    
    MAE_StandardSplit_df = pd.DataFrame([{'CV':n, 'Datentyp': 'Standardsplit', 'Error' : 'MAE', 'X': mae_X, 'y': mae_Y, 'phi': mae_phi}])
    Liste_MAEs_Standardsplit.append(MAE_StandardSplit_df)
    
    df_Fehler.insert(loc=0, column='SplitMethode', value='Standardsplit')
    df_Fehler.insert(loc=1, column='CV', value=n)
    Liste_Fehler_Standardsplit.append(df_Fehler)
    
MAE_Standardsplit = pd.concat(Liste_MAEs_Standardsplit, ignore_index=True)
Fehler_Standardsplit_df = pd.concat(Liste_Fehler_Standardsplit, ignore_index=True)

# for n in Random_numbers:
    
#     X_train, X_val, X_test, Y_train_scaled, Y_val_scaled, Y_test_scaled, Y_train, Y_val, Y_test, scalers_features, scaler_labels, Angepasste_Blechnummern_test   = Fensterung_Scale(data, Validation_data=1, random=n, Train_Test_Split =2, size=0.2)
#     mae_X, mae_Y, mae_phi, df_Fehler = bestes_model(X_train, X_val, X_test, Y_train_scaled, Y_val_scaled, Y_test_scaled, Y_train, Y_val, Y_test, scalers_features, scaler_labels)
    
#     MAE_BlechSplit_df = pd.DataFrame([{'CV':n, 'Datentyp': 'Blechsplit', 'Error' : 'MAE', 'X': mae_X, 'y': mae_Y, 'phi': mae_phi}])
#     Liste_MAEs_Blechsplit.append(MAE_BlechSplit_df)
    
#     df_Fehler.insert(loc=0, column='SplitMethode', value='Blechsplit')
#     df_Fehler.insert(loc=1, column='CV', value=n)
#     Liste_Fehler_Blechsplit.append(df_Fehler)
    
# MAE_Blechsplit = pd.concat(Liste_MAEs_Blechsplit, ignore_index=True)
# Fehler_Blechsplit_df = pd.concat(Liste_Fehler_Blechsplit, ignore_index=True)
    
    
# Ausgabe der DataFrames
print("MAE Standard Split:")
print(MAE_StandardSplit_df)
# print("\nMAE Blech Split:")
# print(MAE_Blechsplit)

[7, 11, 10, 46, 21, 94, 85, 39, 32, 77]
[77]
Anzahl der Zeilen im finalen DataFrame: 255600
None
142
254180
(25418, 1, 3)
Shape für die Features der gesamten Trainingsdaten, also im Falle einer Interpolation mit allen Daten (203344, 10, 11)
Shape für die Features der gesamten Testdaten, also im Falle einer Interpolation mit allen Daten (25418, 10, 11) 
Shape für die Labels der gesamten Trainingsdaten, also im Falle einer Interpolation mit allen Daten (203344, 1, 3)
Shape für die Labels der gesamten Testdaten, also im Falle einer Interpolation mit allen Daten (25418, 1, 3)

Epoch 1/20
6355/6355 ━━━━━━━━━━━━━━━━━━━━ 228s 35ms/step - Verstellweg_Phi_mae: 0.8064 - Verstellweg_X_mae: 0.1760 - Verstellweg_Y_mae: 0.5928 - loss: 3.3654 - val_Verstellweg_Phi_mae: 0.2892 - val_Verstellweg_X_mae: 0.0198 - val_Verstellweg_Y_mae: 0.1849 - val_loss: 1.1476
Epoch 2/20
6355/6355 ━━━━━━━━━━━━━━━━━━━━ 220s 35ms/step - Verstellweg_Phi_mae: 0.2595 - Verstellweg_X_mae: 0.0161 - Verstellweg_Y_mae: 0.1833 - 

In [ ]:
print()

In [3]:
MAE_Standardsplit = pd.concat(Liste_MAEs_Standardsplit, ignore_index=True)

Mean_Standard = MAE_Standardsplit[['X','y','phi']].mean() 
Mean_Blech = MAE_BlechSplit_leer[['X','y','phi']].mean() 
Std_Standard = MAE_Standardsplit[['X','y','phi']].std()
Std_Blech = MAE_BlechSplit_leer[['X','y','phi']].std()

MAE_StandardSplit_df = pd.concat([MAE_Standardsplit, pd.DataFrame([{'CV': 'Mittelwert', 'Datentyp': 'Standardsplit', 'Error' : 'MAE', 'X': Mean_Standard[0], 'y': Mean_Standard[1], 'phi': Mean_Standard[2]}])], ignore_index=True)
MAE_BlechSplit_df = pd.concat([MAE_BlechSplit_leer, pd.DataFrame([{'CV': 'Mittelwert', 'Datentyp': 'Blechsplit', 'Error' : 'MAE', 'X': Mean_Blech[0], 'y': Mean_Blech[1], 'phi': Mean_Blech[2]}])], ignore_index=True)
MAE_StandardSplit_comp = pd.concat([MAE_StandardSplit_df, pd.DataFrame([{'CV': 'Standardabweichung', 'Datentyp': 'Standardsplit', 'Error' : 'MAE', 'X': Std_Standard[0], 'y': Std_Standard[1], 'phi': Std_Standard[2]}])], ignore_index=True)
MAE_BlechSplit_comp = pd.concat([MAE_BlechSplit_df, pd.DataFrame([{'CV': 'Standardabweichung', 'Datentyp': 'Blechsplit', 'Error' : 'MAE', 'X': Std_Blech[0], 'y': Std_Blech[1], 'phi': Std_Blech[2]}])], ignore_index=True)

# print(MAE_BlechSplit)
# print(MAE_StandardSplit)

Errors_for_CSV = pd.concat([MAE_StandardSplit_comp, MAE_BlechSplit_comp], axis=1)
print(Errors_for_CSV['X'])

Errors_for_CSV.columns = ['CV', 'Datentyp', 'Error', 'X', 'y', 'phi', 'CV1',
       'Datentyp1', 'Error1', 'X1', 'y1', 'phi1']

for Column in Errors_for_CSV.columns:
        Errors_for_CSV[Column] = Errors_for_CSV[Column].astype(str).str.replace('.', ',')
        
# for Column in Fehler_Blechsplit_df:
#         Fehler_Blechsplit_df[Column] = Fehler_Blechsplit_df[Column].astype(str).str.replace('.', ',')

Ordner = r'C:\Users\corvi\OneDrive - stud.tu-darmstadt.de\Desktop\Masterthesis\13_ExcelvonDaten_Code\DeepLearning\LSTM\RandomSeed2'

# Fehler_Blechsplit_df.to_csv(f'{Ordner}\Fehler_LSTM_Blechsplit_RandomSeed2_Modell1.csv', index=True, sep=';')

Errors_for_CSV.to_csv(f'{Ordner}\Ergebnisse_LSTM_Standardsplit_RandomSeed2_Modell1[10].csv', index=True, sep=';')

          X    X
0  0.003634  NaN
1  0.003634  NaN
2       NaN  NaN


C:\Users\corvi\AppData\Local\Temp\ipykernel_9772\1955358810.py:23: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Errors_for_CSV[Column] = Errors_for_CSV[Column].astype(str).str.replace('.', ',')
